In [ ]:
 ## start


#print("Hello, World!")

Hello, World!


In [2]:
# Import packages
import os
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer

#import pyarrow.parquet as pq # https://arrow.apache.org/docs/python/parquet.html #error on install

import sys
print(sys.executable)


/opt/anaconda3/envs/nt_bird_detect_conda_env_20251206/bin/python


In [ ]:
recordings_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/20251122_Recordings/Data' #154 files
#recordings_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/20250907_Recordings/Data' #176 files
#recordings_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/20251209_trials_5_recordings/Data' # 5 files
parquet_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/recordings_parquet/recordings_20251211.parquet'


entries = sorted(os.listdir(recordings_directory_path))

# Initialize the BirdNET-Analyzer.
analyzer = Analyzer()

error_files = []
counter_files = 0
for file in entries:
    counter_files += 1
    if not file.lower().endswith('.wav'): continue # skip non-wav files
    directory_path_file = os.path.join(recordings_directory_path, file)
    print(f"Analyzing file number {counter_files}: {file}")
    file_date = file[9:17]
    file_time = file[18:24]
    file_year = int(file[9:13])
    file_month = int(file[13:15])
    file_day = int(file[15:17])
    print(f"Date: {file_date}, Time: {file_time}, Year: {file_year}, Month: {file_month}, Day: {file_day}")

    recording = Recording(
        analyzer,
        directory_path_file,
        lat=50.9481,
        lon=-3.2503,
        date=datetime(year=file_year, month=file_month, day=file_day),  # use date or week_48
        min_conf=0.01,
    )
    
    print(f"Analyzing file: {file}")
    try:
        recording.analyze()
    except Exception as e:
        error_files.append(file) # append errored file name to error list
        print(f"Error analyzing file {file}: {e}")
        continue

    #print("Load data into DataFrame")
    df_recordings_results = pd.DataFrame(recording.detections)

    # Add file name and date columns to DataFrame
    df_recordings_results['file_name'] = file
    df_recordings_results['file_date'] = file_date
    df_recordings_results['file_time'] = file_time

    #print("If Parquet file exists, read it and concatenate, else use current DataFrame")
    if os.path.exists(parquet_directory_path):
        df_stored_recordings = pd.read_parquet(parquet_directory_path)
        #print("Combine both DataFrames")
        df_combined = pd.concat([df_stored_recordings, df_recordings_results], ignore_index=True)
        df = df_combined
    else:
        df = df_recordings_results

    # Save the DataFrame to a Parquet file
    # index=False tells Pandas NOT to save the row indices (0, 1, 2...) as a column
    #print("Save combined DataFrame back to Parquet")
    try:
        df.to_parquet(parquet_directory_path, index=False)
    except ImportError:
        print("Error: Failed to save DataFrame to Parquet")


    #print(directory_path_file)
    #print(file_date, file_time, file_year, file_month, file_day)







Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model


INFO: Applying 1 TensorFlow Lite delegate(s) lazily.
VERBOSE: XNNPack weight cache not enabled.
VERBOSE: Replacing 245 out of 376 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 40 partitions for the whole graph.
INFO: Successfully applied the default TensorFlow Lite delegate indexed at 0.
 *NOTE*: because a delegate has been applied, the precision of computations should be unchanged, but the exact output tensor values may have changed. If such output values are checked in your code, like in your tests etc., please consider increasing error tolerance for the check.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.
VERBOSE: XNNPack weight cache not enabled.
VERBOSE: Replacing 39 out of 56 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 7 partitions for the whole graph.
INFO: Successfully applied the default TensorFlow Lite delegate indexed at 0.
 *NOTE*: because a delegate has been applied, the precision of computations should be unchanged, but the exact out

Meta model loaded.
Analyzing file number 1: S4A27301_20250917_103013.wav
Date: 20250917, Time: 103013, Year: 2025, Month: 9, Day: 17
Analyzing file: S4A27301_20250917_103013.wav
read_audio_data

Error analyzing file S4A27301_20250917_103013.wav: Audio format could not be opened.
Analyzing file number 2: S4A27301_20250917_161252.wav
Date: 20250917, Time: 161252, Year: 2025, Month: 9, Day: 17
Analyzing file: S4A27301_20250917_161252.wav
read_audio_data
read_audio_data: complete, read  1199 chunks.
analyze_recording S4A27301_20250917_161252.wav
recording has lon/lat
set_predicted_species_list_from_position
return_predicted_species_list
35
141 species loaded.
Analyzing file number 3: S4A27301_20250917_171252.wav
Date: 20250917, Time: 171252, Year: 2025, Month: 9, Day: 17
Analyzing file: S4A27301_20250917_171252.wav
read_audio_data
read_audio_data: complete, read  199 chunks.
analyze_recording S4A27301_20250917_171252.wav
recording has lon/lat
set_predicted_species_list_from_position
Analyz

In [5]:
recordings_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/20251122_Recordings/Data' #154 files
#recordings_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/20250907_Recordings/Data' #176 files
#recordings_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/20251209_trials_5_recordings/Data' # 5 files
parquet_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/recordings_parquet/recordings_20251211.parquet'


entries = sorted(os.listdir(recordings_directory_path))

print(entries)

['S4A27301_20250917_103013.wav', 'S4A27301_20250917_161252.wav', 'S4A27301_20250917_171252.wav', 'S4A27301_20250918_045300.wav', 'S4A27301_20250918_055302.wav', 'S4A27301_20250918_065302.wav', 'S4A27301_20250918_075302.wav', 'S4A27301_20250918_085302.wav', 'S4A27301_20250918_095303.wav', 'S4A27301_20250918_105302.wav', 'S4A27301_20250918_115302.wav', 'S4A27301_20250918_125302.wav', 'S4A27301_20250918_135302.wav', 'S4A27301_20250918_145302.wav', 'S4A27301_20250918_155303.wav', 'S4A27301_20250918_165302.wav', 'S4A27301_20250919_045400.wav', 'S4A27301_20250919_055402.wav', 'S4A27301_20250919_065402.wav', 'S4A27301_20250919_075402.wav', 'S4A27301_20250919_085402.wav', 'S4A27301_20250919_095403.wav', 'S4A27301_20250919_105402.wav', 'S4A27301_20250919_115402.wav', 'S4A27301_20250919_125402.wav', 'S4A27301_20250919_135402.wav', 'S4A27301_20250919_145402.wav', 'S4A27301_20250919_155403.wav', 'S4A27301_20250919_165402.wav', 'S4A27301_20250920_045600.wav', 'S4A27301_20250920_055602.wav', 'S4A273

In [ ]:

parquet_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/recordings_parquet/recordings_20251211.parquet'

df_stored_recordings = pd.read_parquet(parquet_directory_path)

#df_stored_recordings.head(20)

df_stored_recordings[df_stored_recordings["confidence"]>0.5].groupby("label").agg(counts=("scientific_name","count")).sort_values(by="counts", ascending=False)



,counts
label,
Corvus frugilegus_Rook,17
Psittacula krameri_Rose-ringed Parakeet,17
Erithacus rubecula_European Robin,8
Corvus corone_Carrion Crow,6
Numenius arquata_Eurasian Curlew,5
Ardea cinerea_Gray Heron,4
Pica pica_Eurasian Magpie,3
Anas platyrhynchos_Mallard,1
Branta canadensis_Canada Goose,1


In [36]:



wave_file_2025 = '/Users/christianarnold/Documents/MyProjects/NatureThrive/20250907_Recordings/Data/S4A27301_20250818_141329.wav'

recordings_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/20251122_Recordings/Data' #154 files
#recordings_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/20250907_Recordings/Data' #176 files
#recordings_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/20251209_trials_5_recordings/Data' # 5 files
parquet_directory_path = '/Users/christianarnold/Documents/MyProjects/NatureThrive/recordings_parquet/recordings_20251210.parquet'


## Get list of files in directory
entries = sorted(os.listdir(recordings_directory_path))

print(entries)

['S4A27301_20250917_103013.wav', 'S4A27301_20250917_161252.wav', 'S4A27301_20250917_171252.wav', 'S4A27301_20250918_045300.wav', 'S4A27301_20250918_055302.wav', 'S4A27301_20250918_065302.wav', 'S4A27301_20250918_075302.wav', 'S4A27301_20250918_085302.wav', 'S4A27301_20250918_095303.wav', 'S4A27301_20250918_105302.wav', 'S4A27301_20250918_115302.wav', 'S4A27301_20250918_125302.wav', 'S4A27301_20250918_135302.wav', 'S4A27301_20250918_145302.wav', 'S4A27301_20250918_155303.wav', 'S4A27301_20250918_165302.wav', 'S4A27301_20250919_045400.wav', 'S4A27301_20250919_055402.wav', 'S4A27301_20250919_065402.wav', 'S4A27301_20250919_075402.wav', 'S4A27301_20250919_085402.wav', 'S4A27301_20250919_095403.wav', 'S4A27301_20250919_105402.wav', 'S4A27301_20250919_115402.wav', 'S4A27301_20250919_125402.wav', 'S4A27301_20250919_135402.wav', 'S4A27301_20250919_145402.wav', 'S4A27301_20250919_155403.wav', 'S4A27301_20250919_165402.wav', 'S4A27301_20250920_045600.wav', 'S4A27301_20250920_055602.wav', 'S4A273

In [ ]:
import pandas as pd
import os

## ca_note: parquet example

# 1. Define the filename for the output
output_file = '/Users/christianarnold/Documents/MyProjects/NatureThrive/bio-acoustic-monitor/standalone_example.parquet'

# 2. Create a simple Pandas DataFrame (the 'df' object)
data = {'A': [1, 2, 3], 
        'B': ['x', 'y', 'z'], 
        'C': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03'])}
df = pd.DataFrame(data)

print("Original DataFrame:\n", df)

# 3. Save the DataFrame to a Parquet file
# index=False tells Pandas NOT to save the row indices (0, 1, 2...) as a column
try:
    df.to_parquet(output_file, index=False)
    print(f"\nSuccessfully saved data to '{output_file}'")
    
    # 4. Verification: Read the data back to confirm it worked
    df_loaded = pd.read_parquet(output_file)
    print("\nDataFrame read back from Parquet:\n", df_loaded)

    # 5. Clean up the created file (optional)
    os.remove(output_file)
    
except ImportError:
    print("\nError: PyArrow or FastParquet library is required but not installed.")
    print("Run: conda install pyarrow")

Original DataFrame:
    A  B          C
0  1  x 2023-01-01
1  2  y 2023-01-02
2  3  z 2023-01-03

Successfully saved data to '/Users/christianarnold/Documents/MyProjects/NatureThrive/bio-acoustic-monitor/standalone_example.parquet'

DataFrame read back from Parquet:
    A  B          C
0  1  x 2023-01-01
1  2  y 2023-01-02
2  3  z 2023-01-03
